In [ ]:
import sys
print(sys.executable)


In [ ]:
import redback
print(redback.__version__)

In [ ]:
import numpy as np
import pandas as pd
from redback.transient_models.phenomenological_models import exponential_powerlaw, fallback_lbol
from redback.transient_models.magnetar_models import magnetar_only, basic_magnetar
from redback.transient_models.magnetar_driven_ejecta_models import _ejecta_dynamics_and_interaction
from redback.transient_models.shock_powered_models import  _shocked_cocoon, _csm_shock_breakout
import redback.interaction_processes as ip
import redback.sed as sed
from redback.sed import flux_density_to_spectrum, blackbody_to_spectrum
import redback.photosphere as photosphere
from astropy.cosmology import Planck18 as cosmo  # noqa
from redback.utils import (calc_kcorrected_properties, citation_wrapper, logger, get_csm_properties, nu_to_lambda,
                           lambda_to_nu, velocity_from_lorentz_factor, build_spectral_feature_list)
from redback.constants import day_to_s, solar_mass, km_cgs, au_cgs, speed_of_light, sigma_sb
from inspect import isfunction
import astropy.units as uu
from collections import namedtuple
from scipy.interpolate import interp1d, RegularGridInterpolator

import matplotlib.pyplot as plt

In [ ]:
#I DID IT FINALLY !! THIA IS MY CLONED REPO FROM GITHUB
#NOW I HAVE TO GET REDBACK IN HERE AND RUN SOME CODE 

#define variables 
#rename tts as time
time = np.geomspace(0.01, 90, 200) 

#amplitude and redshift are defined at the end for plotting 

def sn1998bw_template(time, redshift, amplitude, **kwargs):
    """
    A wrapper to the SN1998bw template. Only valid between 1100-11000 Angstrom and 0.01 to 90 days post explosion in rest frame

    Parameters
    ----------
    time
        time in days in observer frame (post explosion)
    redshift
        redshift
    amplitude
        amplitude scaling factor, where 1.0 is the original brightness of SN1998bw; and f_lambda is scaled by this factor
    kwargs
        Additional keyword arguments required by redback.
    frequency
        Required if output_format is 'flux_density'. frequency to calculate - Must be same length as time array or a single number).
    bands
        Required if output_format is 'magnitude' or 'flux'.
    output_format
        'flux_density', 'magnitude', 'spectra', 'flux', 'sncosmo_source'
    cosmology
        Cosmology to use for luminosity distance calculation. Defaults to Planck18. Must be a astropy.cosmology object.
    Returns
    -------
        set by output format - 'flux_density', 'magnitude', 'spectra', 'flux', 'sncosmo_source'

    """
    import sncosmo
    model = sncosmo.Model(source='v19-1998bw')
    original_redshift = 0.0085
    cosmology = kwargs.get("cosmology", cosmo)
    original_dl = (43*uu.Mpc).to(uu.cm).value

    # From roughly matching to Galama+ or Clocchiatti+1998bw light curves
    original_peak_time = 15
    model.set(z=original_redshift, t0=original_peak_time)
    model.set_source_peakmag(14.25, band='bessellb', magsys='ab')
    lls = np.linspace(1620, 11000, 300)
    f_lambda = model.flux(time, lls) #erg/s/cm^2/Angstrom.
    l_lambda = f_lambda * 4 * np.pi * original_dl**2  # erg/s/Angstrom

    # We consider this the rest frame spectrum of 1998bw. Now we can redshift it and scale it.
    time_obs = time * (1 + redshift)
    lambda_obs = lls * (1 + redshift)
    dl_new = cosmology.luminosity_distance(redshift).cgs.value
    f_lambda_obs = l_lambda / (4 * np.pi * dl_new**2)
    f_lambda_obs = amplitude * f_lambda_obs * (1 + redshift) # accounting for bandwidth stretching
    f_lambda_obs = f_lambda_obs * uu.erg / uu.s / uu.cm ** 2 / uu.Angstrom
    if kwargs['output_format'] == 'flux_density':
        frequency = kwargs['frequency']
        # work in obs frame
        ff_array = lambda_to_nu(lambda_obs)

        # Convert flux density to mJy
        fmjy = f_lambda_obs.to(uu.mJy, equivalencies=uu.spectral_density(wav=lambda_obs * uu.Angstrom)).value
        # Create interpolator on obs frame grid
        flux_interpolator = RegularGridInterpolator(
            (time_obs, ff_array),
            fmjy,
            bounds_error=False,
            fill_value=0.0)

        # Prepare points for interpolation
        if isinstance(frequency, (int, float)):
            frequency = np.ones_like(time) * frequency

        # Create points for evaluation
        points = np.column_stack((time, frequency))

        # Return interpolated flux density with (1+z) correction for observer frame
        return flux_interpolator(points)
    elif kwargs['output_format'] == 'spectra':
        return namedtuple('output', ['time', 'lambdas', 'spectra'])(time=time_obs,
                                                                    lambdas=lambda_obs,
                                                                    spectra=f_lambda_obs)
    else:
        return sed.get_correct_output_format_from_spectra(time=time, time_eval=time_obs,
                                                          spectra=f_lambda_obs, lambda_array=lambda_obs,
                                                          **kwargs)
    
mag = sn1998bw_template(time, redshift=1, amplitude=0.001, output_format='magnitude', bands='bessellb')
plt.plot(time, mag)
plt.xlabel("Time")
plt.ylabel("Magnitude")
plt.gca().invert_yaxis() #lower magnitudes = brighter
plt.show()

#why is y axis so large ??
#for extinction can add extinction_corrected = True - is this a thing in redback ? 

In [ ]:
#now compare to data from files and plot against the redback output 

sn1998bw_z_0_8 = pd.read_csv('/Users/helenagrabham/Downloads/98_bw_models/98bw_z0.8.txt')

sn1998bw_z_0_8.to_csv('/Users/helenagrabham/Downloads/98_bw_models/98bw_z0.8.csv', index = False)

print(sn1998bw_z_0_8)

#compare this data to the model to ensure model is behaving correctly 
#OR ! imputting the data into this model 

#choose r-band as comparison 
#compare two lots of data files from around z = 0.8 and see how they match with the redback model

time = sn1998bw_z_0_8['day']
mag_r = sn1998bw_z_0_8['r']

plt.plot(time, mag_r, '-', label = 'Observed r-band data')
plt.xlabel("Time")
plt.ylabel("Magnitude")
plt.gca().invert_yaxis() #lower magnitudes = brighter
plt.show()

In [ ]:
#now plot multiple on the same lightcurve plot 
#redshifts need to be the same here !! decrease / increase amplitude 

# Load data
data = pd.read_csv('/Users/helenagrabham/Downloads/98_bw_models/98bw_z1.0.txt', comment='#')
time = data['day']
mag_r = data['r']

mag_i = data['i']

# Model output for all bands ? what does bessellb mean B BAND !! found in tables/filters.csv 
# mag_model = sn1998bw_template(
#     time,
#     redshift=0.8,
#     amplitude=1.0,  # adjust as needed
#     output_format='magnitude',
#     bands='bessellb' #change this ? 
# )

mag_model = sn1998bw_template(
    time,
    redshift=1.0,
    amplitude=1.2,  # adjust as needed (if model too faint increase amplitude and vice versa)
    output_format='magnitude',
    bands='bessellb' #change this ? 
)
# Plot
plt.plot(time, mag_r, '-', label='Observed r-band')
plt.plot(time, mag_i, '-', label='Observed i-band')
plt.plot(time, mag_model, '-', label='Model r-band')
plt.gca().invert_yaxis()
plt.xlabel('Time (days)')
plt.ylabel('Magnitude')
plt.legend()
plt.show()

#is this extinction corrected in redback ? need to add it in ! simply done in redback apparently ? 

#working with apparent magnitudes 


In [ ]:
#this plot gives us the best model params we need to compare SN 1998bw model to other events ? YES but must be same redshift obs - model
#to make more comparable change the amplitude param 
#also have to apply extinction - HOW 
#just apply extinction correction to model ? apply host galaxy extinction - is data already extinction corrected? ask gavin 

#sometimes amplitude is artificially low because model thinks SN is naturally dim when really it is hidden behind dust 

'''attempt a calculation without the extinction correction and then if doesnt work see where to go from gavin'''

In [ ]:
#now compare to an event: 060218 SN 2006aj apparent magnitude = 17.22 in the r-band MUST BE ASSOCIATED WITH A SPECIFIC EPOCH 11 days 

import numpy as np
from redback.transient_models.supernova_models import sn1998bw_template

# Epoch we want to compare (e.g., 20 days post-explosion)
epoch = 11.0
z_event = 0.0331 # Redshift of your new GRB

# 1. Get 1998bw magnitude at this epoch
m_98bw = sn1998bw_template(t=epoch, redshift=z_event, amplitude=1.0, 
                           output_format='magnitude', bands='bessellb')

# 2. Get your new GRB's magnitude at this epoch (from your Redback fit)
# Let's say your fit found amplitude = 0.8
m_grb = sn1998bw_template(t=epoch, redshift=z_event, amplitude=0.8, 
                          output_format='magnitude', bands='bessellb')

# 3. Calculate Ratio
mag_diff = m_grb - m_98bw
flux_ratio = 10**(-0.4 * mag_diff)

print(f"At {epoch} days, the GRB is {flux_ratio:.2f}x as bright as SN 1998bw.")

In [ ]:
#i already know the apparent magnitude of the event so use it !! 
import numpy as np
# Ensure you use the custom function you defined in your previous script
# from redback.transient_models.supernova_models import sn1998bw_template

# 1. The actual data for SN 2006aj
epoch = 11.0
z_event = 0.0331
m_obs_2006aj = 17.22  # The r-band apparent magnitude you provided

# 2. Get the 1998bw "Standard" magnitude at THIS epoch and THIS redshift
# We use amplitude=1.0 to get the 'baseline'
m_98bw_baseline = sn1998bw_template(
    time=epoch, 
    redshift=z_event, 
    amplitude=1.0, 
    output_format='magnitude', 
    bands='sdssr' # Match the r-band of your observation
)

# 3. Calculate Ratio
# mag_diff = Observed - Reference
mag_diff = m_obs_2006aj - m_98bw_baseline
flux_ratio = 10**(-0.4 * mag_diff)

print(f"At {epoch} days (r-band):")
print(f"SN 1998bw Model Magnitude: {m_98bw_baseline:.2f}")
print(f"SN 2006aj Observed Magnitude: {m_obs_2006aj:.2f}")
print(f"Flux Ratio: {flux_ratio:.4f}")

#THIS WORKED !! TRY FOR ANOTHER WHICH I KNOW TO BE TRUE THEN CAN CALCULATE THE REST OF THEM 

In [ ]:
#trial for event 2: use the most well known result : GRB 190829A

# 1. The actual data for 190829A - not the same band ! cant find r-band data 
epoch =  12.35
z_event = 0.0785
m_obs_190829A = 21.2  # The r-band apparent magnitude you provided

# 2. Get the 1998bw "Standard" magnitude at THIS epoch and THIS redshift
# We use amplitude=1.0 to get the 'baseline'
m_98bw_baseline = sn1998bw_template(
    time=epoch, 
    redshift=z_event, 
    amplitude=1.0, 
    output_format='magnitude', 
    bands='sdssr' # Match the r-band of your observation
)

# 3. Calculate Ratio
# mag_diff = Observed - Reference
mag_diff = m_obs_190829A - m_98bw_baseline
flux_ratio = 10**(-0.4 * mag_diff)

print(f"At {epoch} days (r-band):")
print(f"SN 1998bw Model Magnitude: {m_98bw_baseline:.2f}")
print(f"GRB 190829A Observed Magnitude: {m_obs_190829A:.2f}")
print(f"Flux Ratio: {flux_ratio:.4f}")

#WELL THIS IS CLEARLY WRONG !! 

In [ ]:
#try one more: 
#ensure bands match up !! at least u,g,r,i,z 

# 1. The actual data for 190829A - not the same band ! cant find r-band data 
epoch =  12.35
z_event = 0.0785
m_obs_190829A = 21.2  # The r-band apparent magnitude you provided

# 2. Get the 1998bw "Standard" magnitude at THIS epoch and THIS redshift
# We use amplitude=1.0 to get the 'baseline'
m_98bw_baseline = sn1998bw_template(
    time=epoch, 
    redshift=z_event, 
    amplitude=1.0, 
    output_format='magnitude', 
    bands='sdssr' # Match the r-band of your observation
)

# 3. Calculate Ratio
# mag_diff = Observed - Reference
mag_diff = m_obs_190829A - m_98bw_baseline
flux_ratio = 10**(-0.4 * mag_diff)

print(f"At {epoch} days (r-band):")
print(f"SN 1998bw Model Magnitude: {m_98bw_baseline:.2f}")
print(f"GRB 190829A Observed Magnitude: {m_obs_190829A:.2f}")
print(f"Flux Ratio: {flux_ratio:.4f}")

#test + send emails tomoror
